# Task 1) Tabular Q-Learning with Taxi-v3 
In this task, you'll implement **Tabular Q-Learning** algorithm. You will use it for the Taxi-v3 Gym environment to learn to **navigate in a city to transport its passengers from point A to point B.**

### Step 0: Install and import the libraries
We will use different libraries:
- `Numpy`: For handling our Q-table.
- `OpenAI Gym`: Contains our 🚕 environment (Taxi-v3).
- `Random`: To generate random numbers (that will be useful for Epsilon-Greedy Policy).


In [2]:
%matplotlib inline
import numpy as np
import gym
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython import display

## Step 1: Create the environment 
- Here we'll create the Taxi-v3 environment. 

Our environment looks like this: 
- It's a **5x5 grid world**
- Our 🚕 is spawned **randomly** in a square. 
- The passenger is **spawned randomly in one of the 4 possible locations** (R, B, G, Y) and wishes to go in one of the **4 possibles locations too**.

The reward system:
- -1 for each timestep
- +20 for successfully deliver the passenger
- -10 for illegal actions (pickup or putdown the passenger at the outside of the destination).

In [3]:
env = gym.make("Taxi-v3")
env.reset()
print(env.render(mode='ansi'))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




## Step 2: Create the Q-table and initialize it
- Now, we'll create our Q-table, to know how much rows (states) and columns (actions) we need to know the **action_space and the observation_space**.
- OpenAI Gym provides us a way to do that: `env.action_space.n` and `env.observation_space.n`

In [4]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")
action_space = env.action_space.n
print("There are ", action_space, " possible actions")

There are  500  possible states
There are  6  possible actions


In [5]:
# Create our Q table with state_size rows and action_size columns (500x6)
Q = np.zeros((state_space, action_space))
print(Q)
print(Q.shape)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
(500, 6)


## Step 3: Define the hyperparameters
- Here, we'll specify the hyperparameters

In [6]:
total_episodes = 25000        # Total number of training episodes
total_test_episodes = 100     # Total number of test episodes
max_steps = 200               # Max steps per episode

learning_rate = 0.01          # Learning rate
gamma = 0.99                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.001            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

## Step 4: Define the epsilon-greedy policy

Epsilon Greedy:

- *With probability 1 - ɛ* : we do **exploitation** (aka our agent selects the action with the highest state-action pair value).

- *With probability ɛ*: we do **exploration** (trying random action).

And as the training goes, **we progressively reduce the epsilon value** since we will **need less and less exploration and more exploitation**.

In [7]:
def epsilon_greedy_policy(Q, state):
  # if random number > greater than epsilon --> exploitation
  if(random.uniform(0,1) > epsilon):
    action = np.argmax(Q[state])
  # else --> exploration
  else:
    action = env.action_space.sample()
  
  return action

## Step 5: Define the Q-Learning algorithm and train our agent
- Now we implement the Q learning algorithm and the training loop:

In [ ]:
 for episode in tqdm(range(total_episodes)):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False

    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    
    for step in range(max_steps):
        #
        action = epsilon_greedy_policy(Q, state)

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        Q[state][action] = Q[state][action] + learning_rate * (reward + gamma * 
                                    np.max(Q[new_state]) - Q[state][action])      
        # If done : finish episode
        if done == True: 
            break
        
        # Our new state is state
        state = new_state

## Step 6: Testing and Visualization
- By running the following cells, you'll see your smart taxi agent.




In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        print(f"Episode: {frame['episode']}")
        sleep(.1)

In [ ]:
import time
from IPython.display import clear_output
rewards = []

frames = []
for episode in tqdm(range(total_test_episodes)):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    print("****************************************************")
    print("EPISODE ", episode)
    for step in range(max_steps):
        env.render()     
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(Q[state][:])
        new_state, reward, done, info = env.step(action)
        total_rewards += reward
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward,
            'episode': episode + 1
            }
        )


        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
env.close()
print_frames(frames)
print ("Average episodic score: " +  str(sum(rewards)/total_test_episodes))